INSERT REPORT TITLE AND INFO HERE

### Imports

In [ ]:
# Imports
import importlib
# mypytable
import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable
# myevaluation
import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as me
# myutils
import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as mu

## Step 1: Data Cleaning

### Preprocessing

The data came as a large number of JSON files grouped into folders. Many of the files and folders were empty, so we began by deleting those. Some of the files contained irrelevant data/non-changing data, such as date of birth and device specs, so we also deleted those. What we were left with was three sets of JSON files, aggregator, fitness, and wellness.

* Aggregator: This contains the most info. It has a list of metrics related to stress, calories, heart rate, and minor metrics related to activity
* Fitness: This contains a list of activities and information about them. All of these activities are runs. Has attributes like distance, speed, heart rate, duration
* Wellness: This mainly contained sleep data

### Cleaning/Joining TODO

Many instances in the dataset are missing values, or are just instances with basically no data. TODO: Make copies of the data without these bad instances

Additionally, for this step we combine all of the many JSON files into one. First by joining all files within each folder, simply appending them onto each other. Then join the lists from each folder. 

We also opened the CSV's in Excel to guide our decision-making process for this step.

In [17]:
# Cleaning/Joining...
# SLEEP
# 1. load all files
full_sleep_table = MyPyTable()
full_sleep_table.load_from_file('csv_converted_data/connect_wellness/2021-04-23_2021-08-01_96200873_sleepData.csv')
other_table_1 = MyPyTable()
other_table_1.load_from_file('csv_converted_data/connect_wellness/2021-08-01_2021-11-09_96200873_sleepData.csv')
other_table_2 = MyPyTable()
other_table_2.load_from_file('csv_converted_data/connect_wellness/2021-11-09_2022-02-17_96200873_sleepData.csv')
other_table_3 = MyPyTable()
other_table_3.load_from_file('csv_converted_data/connect_wellness/2022-02-17_2022-05-28_96200873_sleepData.csv')
other_table_4= MyPyTable()
other_table_4.load_from_file('csv_converted_data/connect_wellness/2022-05-28_2022-09-05_96200873_sleepData.csv')
other_table_5 = MyPyTable()
other_table_5.load_from_file('csv_converted_data/connect_wellness/2022-09-05_2022-12-14_96200873_sleepData.csv')
other_table_6 = MyPyTable()
other_table_6.load_from_file('csv_converted_data/connect_wellness/2022-12-14_2023-03-24_96200873_sleepData.csv')
other_table_7 = MyPyTable()
other_table_7.load_from_file('csv_converted_data/connect_wellness/2023-03-24_2023-07-02_96200873_sleepData.csv')
other_table_8 = MyPyTable()
other_table_8.load_from_file('csv_converted_data/connect_wellness/2023-07-02_2023-10-10_96200873_sleepData.csv')
other_table_9 = MyPyTable()
other_table_9.load_from_file('csv_converted_data/connect_wellness/2023-10-10_2024-01-18_96200873_sleepData.csv')
other_table_10 = MyPyTable()
other_table_10.load_from_file('csv_converted_data/connect_wellness/2024-01-18_2024-04-27_96200873_sleepData.csv')
other_table_11 = MyPyTable()
other_table_11.load_from_file('csv_converted_data/connect_wellness/2024-04-27_2024-08-05_96200873_sleepData.csv')
other_table_12 = MyPyTable()
other_table_12.load_from_file('csv_converted_data/connect_wellness/2024-08-05_2024-11-13_96200873_sleepData.csv')

# 2. Join all files to full
other_tables = [other_table_1,other_table_2,other_table_3,other_table_4,
                other_table_5,other_table_6,other_table_7,other_table_8,
                other_table_9,other_table_10,other_table_11,other_table_12]
for table in other_tables:
    full_sleep_table.data+=table.data
#current_table.pretty_print()
# 3. Basic cleaning
full_sleep_table.remove_rows_with_missing_values()
full_sleep_table.remove_rows_where_col_equal_specified(3,'OFF_WRIST')

# This is all of our joined sleep data
#full_sleep_table.pretty_print()
full_sleep_table.save_to_file('joined_nullfree_subsets/full_sleep.csv')

In [ ]:
# Repeat for other file sets
# ACTIVITY
full_activity_table = MyPyTable()
full_activity_table.load_from_file('csv_converted_data/connect_fitness/jack-brandt@comcast.net_0_summarizedActivities.csv')
other_activity_table = MyPyTable()
other_activity_table.load_from_file('csv_converted_data/connect_fitness/jack-brandt@comcast.net_1001_summarizedActivities.csv')
full_activity_table.data += other_activity_table.data
#full_activity_table.pretty_print()
full_activity_table.save_to_file('joined_nullfree_subsets/full_activity.csv')

  activityId       uuidMsb       uuidLsb  name                   activityType      userProfileId    timeZoneId    beginTimestamp    eventTypeId  rule         sportType            startTimeGmt    startTimeLocal          duration          distance  elevationGain       elevationLoss         avgSpeed  maxSpeed             avgHr    maxHr    avgRunCadence    maxRunCadence    steps      calories  bmrCalories          startLongitude       startLatitude       avgStrideLength       avgFractionalCadence  maxFractionalCadence    vO2MaxValue           elapsedDuration  movingDuration      deviceId            minElevation         maxElevation        avgDoubleCadence    maxDoubleCadence    locationName          maxVerticalSpeed      manufacturer    lapCount             endLongitude         endLatitude         maxLatitude         maxLongitude         minLatitude         minLongitude         decoDive    purposeful    autoCalcCalories    favorite    pr     elevationCorrected    atpActivity    parent    a

In [20]:
# Repeat for other file sets
# AGGREGATOR
full_aggregator_table = MyPyTable()
full_activity_table.load_from_file('csv_converted_data/connect_aggregator_data/UDSFile_2021-04-23_2021-08-01.csv')
other_aggregator_1 = MyPyTable()
other_aggregator_1.load_from_file('csv_converted_data/connect_aggregator_data/UDSFile_2021-08-01_2021-11-09.csv')
other_aggregator_2 = MyPyTable()
other_aggregator_2.load_from_file('csv_converted_data/connect_aggregator_data/UDSFile_2021-11-09_2022-02-17.csv')
other_aggregator_3 = MyPyTable()
other_aggregator_3.load_from_file('csv_converted_data/connect_aggregator_data/UDSFile_2022-02-17_2022-05-28.csv')
other_aggregator_4 = MyPyTable()
other_aggregator_4.load_from_file('csv_converted_data/connect_aggregator_data/UDSFile_2022-05-28_2022-09-05.csv')
other_aggregator_5 = MyPyTable()
other_aggregator_5.load_from_file('csv_converted_data/connect_aggregator_data/UDSFile_2022-09-05_2022-12-14.csv')
other_aggregator_6 = MyPyTable()
other_aggregator_6.load_from_file('csv_converted_data/connect_aggregator_data/UDSFile_2022-12-14_2023-03-24.csv')
other_aggregator_7 = MyPyTable()
other_aggregator_7.load_from_file('csv_converted_data/connect_aggregator_data/UDSFile_2023-03-24_2023-07-02.csv')
other_aggregator_8 = MyPyTable()
other_aggregator_8.load_from_file('csv_converted_data/connect_aggregator_data/UDSFile_2023-07-02_2023-10-10.csv')
other_aggregator_9 = MyPyTable()
other_aggregator_9.load_from_file('csv_converted_data/connect_aggregator_data/UDSFile_2023-10-10_2024-01-18.csv')
other_aggregator_10 = MyPyTable()
other_aggregator_10.load_from_file('csv_converted_data/connect_aggregator_data/UDSFile_2024-01-18_2024-04-27.csv')
other_aggregator_11 = MyPyTable()
other_aggregator_11.load_from_file('csv_converted_data/connect_aggregator_data/UDSFile_2024-04-27_2024-08-05.csv')
other_aggregator_12 = MyPyTable()
other_aggregator_12.load_from_file('csv_converted_data/connect_aggregator_data/UDSFile_2024-08-05_2024-11-13.csv')

# 2. Join all files to full
other_tables = [other_aggregator_1,other_aggregator_2,other_aggregator_3,other_aggregator_4,
                other_aggregator_5,other_aggregator_6,other_aggregator_7,other_aggregator_8,
                other_aggregator_9,other_aggregator_10,other_aggregator_11,other_aggregator_12]
for table in other_tables:
    full_aggregator_table.data+=table.data

full_aggregator_table.pretty_print()
full_sleep_table.save_to_file('joined_nullfree_subsets/full_aggregator.csv')

-----------  ----------  --------------------------------  ---------  ----  ----  ----  ----  ----  ----  ----  -------  -----  -----  -----  ---------------------  ---------------------  ---------------------  ---------------------  ---------------------  -----  -----  ---  ---  -----  -----  -----  -----  -----  ---------------  ---------------  ---------------  -----  ----------  ----------  -----  ----------  ----------  ----------  ----------  ----------  ----------  ------  -------  -------  ------  ------  -------  --------  --------  --------  -------  ----------  -------  -------  -------  -------  ---------------  ---------------  ------  -------  --------  --------  -------  -------  --------  --------  --------  -------  -------  -------  --------  --------  -------  -------  ----------  ----------  -------  ----------  ----------  ----------  -----  --------  ---------------------  -------  --------  ---------------------  ---------------------  ----------  ----------  ---

In [ ]:
# More joining/Cleaning here
#?

In [ ]:
# Load data and discretize!

# Save discretized data?


In [ ]:
#Set up models # Add more from myclassifiers if time
dummy_model = MyDummyClassifier() # Import these from myclassifiers
knn_model = MyKNeighborsClassifier()
bayes_model = MyNaiveBayesClassifier()
tree_model = MyDecisionTreeClassifier()
forest_model = MyRandomForestClassifier()

# Set up X
final_table = MyPyTable()
final_table.load_from_file('?')
X = final_table.get_data_subset(['INSERT IMPORTANT ATTRIBUTES HERE'],False)
y = final_table.get_column('DISCRETIZED Y COLUMN HERE')
# Labels
labels = ['slow','mild','fast']
# Positive Label
pos_label='fast'

# Repeat these following two lines for each possible model
metrics, confusion, clas_repor = me.get_metrics_and_conf_matrix_and_report(dummy_model,10,X,y,labels,pos_label,'Running')
mu.report_metrics_and_confusion('Dummy',metrics, confusion,
    clas_repor)

In [ ]:
metrics, confusion, clas_repor = me.get_metrics_and_conf_matrix_and_report(knn_model,10,X,y,labels,pos_label,'Running')
mu.report_metrics_and_confusion('KNN',metrics, confusion,
    clas_repor)

In [ ]:
metrics, confusion, clas_repor = me.get_metrics_and_conf_matrix_and_report(tree_model,10,X,y,labels,pos_label,'Running')
mu.report_metrics_and_confusion('Tree',metrics, confusion,
    clas_repor)

In [ ]:
metrics, confusion, clas_repor = me.get_metrics_and_conf_matrix_and_report(forest_model,10,X,y,labels,pos_label,'Running')
mu.report_metrics_and_confusion('Forest',metrics, confusion,
    clas_repor)